In [1]:
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, ToolMessage

In [2]:
from langchain_ollama import ChatOllama 

llm = ChatOllama(
    model= "llama3.1:8b", 
    temperature=0  # Low temp for reliable tool calling
)

In [3]:
@tool
def add(a: int, b: int) -> int:
    """
    Add a and b.
    
    Args:
        a (int): first integer to be added
        b (int): second integer to be added

    Return:
        int: sum of a and b
    """
    return a + b

In [4]:
tools = [add]

llm_with_tools = llm.bind_tools(tools)

In [5]:
@tool
def subtract(a: int, b:int) -> int:
    """Subtract b from a."""
    return a - b

@tool
def multiply(a: int, b:int) -> int:
    """Multiply a and b."""
    return a * b

@tool
def divide(a: int, b:int) -> int:
    """divide a by b."""
    return a / b

In [6]:
tool_map = {
    "add": add, 
    "subtract": subtract,
    "multiply": multiply,
    "divide":divide
}

input_ = {
    "a": 1,
    "b": 2
}

tool_map["divide"].invoke(input_)

0.5

In [7]:
tools = [add, subtract, multiply, divide]

llm_with_tools = llm.bind_tools(tools)

In [8]:
query = "What is 3 + 2?"
chat_history = [HumanMessage(content=query)]

In [9]:
response_1 = llm_with_tools.invoke(chat_history)
chat_history.append(response_1)

In [10]:
type(response_1)

langchain_core.messages.ai.AIMessage

In [11]:
response_1

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:8b', 'created_at': '2026-01-17T16:39:57.053960589Z', 'done': True, 'done_reason': 'stop', 'total_duration': 18852935266, 'load_duration': 10049017186, 'prompt_eval_count': 343, 'prompt_eval_duration': 4807006353, 'eval_count': 22, 'eval_duration': 3871047379, 'logprobs': None, 'model_name': 'llama3.1:8b', 'model_provider': 'ollama'}, id='lc_run--019bccd3-99d6-7c21-a0e0-c2c8c3acc48a-0', tool_calls=[{'name': 'add', 'args': {'a': 3, 'b': 2}, 'id': 'b57fb8fe-d70d-4dce-82b1-6c460560d031', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 343, 'output_tokens': 22, 'total_tokens': 365})

In [12]:
tool_calls_1 = response_1.tool_calls

tool_1_name = tool_calls_1[0]["name"]
tool_1_args = tool_calls_1[0]["args"]
tool_call_1_id = tool_calls_1[0]["id"]

print(f'tool name:\n{tool_1_name}')
print(f'tool args:\n{tool_1_args}')
print(f'tool call ID:\n{tool_call_1_id}')

tool name:
add
tool args:
{'a': 3, 'b': 2}
tool call ID:
b57fb8fe-d70d-4dce-82b1-6c460560d031


In [13]:
tool_response = tool_map[tool_1_name].invoke(tool_1_args)
tool_message = ToolMessage(content=tool_response, tool_call_id=tool_call_1_id)

tool_message

ToolMessage(content='5', tool_call_id='b57fb8fe-d70d-4dce-82b1-6c460560d031')

In [14]:
chat_history.append(tool_message)

In [15]:
answer = llm_with_tools.invoke(chat_history)

In [16]:
print(type(answer))
print(answer.content)

<class 'langchain_core.messages.ai.AIMessage'>
The answer to the equation 3 + 2 is 5.


In [17]:
class ToolCallingAgent:
    def __init__(self, llm, tools):
        self.llm_with_tools = llm.bind_tools(tools)
        self.tool_map = tool_map

    def run(self, query: str) -> str:
        # Step 1: Initial user message
        chat_history = [HumanMessage(content=query)]

        # Step 2: LLM chooses tool
        response = self.llm_with_tools.invoke(chat_history)
        if not response.tool_calls:
            return response.contet # Direct response, no tool needed
        # Step 3: Handle first tool call
        tool_call = response.tool_calls[0]
        tool_name = tool_call["name"]
        tool_args = tool_call["args"]
        tool_call_id = tool_call["id"]

        # Step 4: Call tool manually
        tool_result = self.tool_map[tool_name].invoke(tool_args)

        # Step 5: Send result back to LLM
        tool_message = ToolMessage(content=str(tool_result), tool_call_id=tool_call_id)
        chat_history.extend([response, tool_message])

        # Step 6: Final LLM result
        final_response = self.llm_with_tools.invoke(chat_history)
        return final_response.content

In [18]:
my_agent = ToolCallingAgent(llm, tools)

print(my_agent.run("one plus 2"))

print(my_agent.run("two minus one"))

print(my_agent.run("three times two"))

print(my_agent.run("eight divided by 4"))

The result of one plus two is three.
The result of 2 minus 1 is 1.
The result of multiplying 3 by 2 is 6.
The result of dividing 8 by 4 is 2.


In [19]:
@tool
def calculate_tip(total_bill: int, tip_percent: int) -> int:
    """Calculate tip"""
    return total_bill * tip_percent * 0.01

inputs = {
    "total_bill": 120,
    "tip_percent": 15
}
calculate_tip.invoke(inputs)


tool_map = {
    "calculate_tip": calculate_tip
}

In [20]:
query = "How much should I tip on $60 at 20%?"

llm_with_tool = llm.bind_tools([calculate_tip])
chat_history = [HumanMessage(content=query)]

response = llm_with_tool.invoke(chat_history)

tool_calls = response.tool_calls
tool_name = tool_calls[0]["name"]
tool_args = tool_calls[0]["args"]
tool_call_id = tool_calls[0]["id"]

tool_response = tool_map[tool_name].invoke(tool_args)
tool_message = ToolMessage(content=tool_response, tool_call_id=tool_call_id)

chat_history.extend([response, tool_message])

result = llm_with_tool.invoke(chat_history)
print(result.content)

The amount you should tip is $12.00.


In [21]:
class TipAgent:
    def __init__(self, llm, tools):
        self.llm_with_tool = llm.bind_tools(tools)
        self.tool_map = tool_map

    def run(self, query: str) -> str:
        chat_history = [HumanMessage(content=query)]
        response = llm_with_tool.invoke(chat_history)

        tool_calls = response.tool_calls
        tool_name = tool_calls[0]["name"]
        tool_args = tool_calls[0]["args"]
        tool_call_id = tool_calls[0]["id"]
        
        tool_response = tool_map[tool_name].invoke(tool_args)
        tool_message = ToolMessage(content=tool_response, tool_call_id=tool_call_id)
        
        chat_history.extend([response, tool_message])
        
        return llm_with_tool.invoke(chat_history).content

In [22]:
tools = [calculate_tip]

In [23]:
agent = TipAgent(llm, tools)
query = "How much should I tip on 900 at 20%?"

agent.run(query)

'The tip for a $900 bill at 20% would be $180.00.'